# LangChain to prompt programmatically in Python

## Load model

In [26]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-search-preview")

model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0xffff40c0a9c0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0xffff3fbbc050>, root_client=<openai.OpenAI object at 0xffff3fbbb800>, root_async_client=<openai.AsyncOpenAI object at 0xffff40c0a3f0>, model_name='gpt-4o-search-preview', model_kwargs={}, openai_api_key=SecretStr('**********'))

## Prompting seralization

### Create and load prompt from markdown

[../prompts/custom/ES.md](../prompts/custom/ES.md)

In [27]:
from pathlib import Path

path_prompt_md = Path('../prompts/custom/ES.md')
with open(path_prompt_md, 'r') as file:
    template = file.read()
    
template

'Busca las últimas noticias de hoy {DATE} de la acción {TICKER}.'

### Create prompt object

In [28]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template=template,
    template_format='f-string',
)

prompt

PromptTemplate(input_variables=['DATE', 'TICKER'], input_types={}, partial_variables={}, template='Busca las últimas noticias de hoy {DATE} de la acción {TICKER}.')

### Save prompt to JSON

In [29]:
path_prompt_json = path_prompt_md.with_suffix('.json')
prompt.save(path_prompt_json)

## Chaining with LLMs

### Define chain

In [30]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | model | StrOutputParser()

chain

PromptTemplate(input_variables=['DATE', 'TICKER'], input_types={}, partial_variables={}, template='Busca las últimas noticias de hoy {DATE} de la acción {TICKER}.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0xffff40c0a9c0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0xffff3fbbc050>, root_client=<openai.OpenAI object at 0xffff3fbbb800>, root_async_client=<openai.AsyncOpenAI object at 0xffff40c0a3f0>, model_name='gpt-4o-search-preview', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

### Preprocess input data

If your prompt requires to use the current date, use the following variable.

Use the `{VARIABLES}` to fill the prompt from [../prompts/custom/ES.md](../prompts/custom/ES.md).

In [31]:
from datetime import datetime
today = datetime.now().strftime('%Y-%m-%d')

today

'2025-04-11'

### Invoke chain with input data

In [32]:
output = chain.invoke({
    'DATE': today,
    'TICKER': 'AAPL',
})

print(output)

Al 11 de abril de 2025, las acciones de Apple Inc. (AAPL) cotizan a 190,42 USD, con una variación de -8,22 USD (-4,13%) respecto al cierre anterior.

##  Información del mercado de valores para Apple Inc (AAPL)
- Apple Inc es un(a) equity en el mercado de USA.
- El precio es 190.42 USD actualmente  with a change of -8.22 USD (-0.04%) from the previous close.
- The latest open price was 189.0 USD and the intraday volume is 121879981.
- The intraday high is 195.04 USD and the intraday low is 183.04 USD.
- The latest trade time is Friday, April 11, 00:15:00 UTC.


En los últimos meses, Apple ha experimentado fluctuaciones en el precio de sus acciones debido a diversos factores:

- **Anuncio de Apple Intelligence**: En junio de 2024, Apple presentó su propio sistema de inteligencia artificial, Apple Intelligence, en colaboración con OpenAI. Tras el anuncio, las acciones de la compañía cayeron casi un 2% el mismo día, posiblemente debido a expectativas no cumplidas por parte de los inversor

## Export output to file

In [33]:
with open('reports/custom/output.md', 'w') as file:
    file.write(output)

## Iterate over a list of items

Use the same chain to iterate over a list of items.

In [34]:
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA']

for ticker in tickers:
    output = chain.invoke({
        'DATE': today,
        'TICKER': ticker,
    })
    
    with open(f'reports/custom/{ticker}.md', 'w') as file:
        file.write(output)